# Семантическая Сегментация. Часть 2.

In [3]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [4]:
import tensorflow as tf
# Импортируем библиотеку TensorFlow — мощный фреймворк для создания и обучения нейронных сетей.
# Включает инструменты для построения моделей, оптимизации, работы с данными и многого другого.


## Модель ASPP

In [5]:
class ASPPBlock(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # 1x1 convolution: kernel size (1,1) — применяет свертку на каждый пиксель отдельно,
        # 256 фильтров — количество выходных каналов, padding='same' — сохраняет размер входа,
        # activation='relu' — функция активации, добавляет нелинейность, помогает модели учить сложные зависимости
        self.conv1 = tf.keras.layers.Conv2D(256, (1, 1), padding='same', activation='relu')

        # 3x3 сверточный слой с dilation_rate=6 (расширенная свертка):
        # dilation_rate — увеличивает размер поля обзора ядра без уменьшения размера выхода,
        # padding='same' — сохраняет размер,
        # активация ReLU — для нелинейности
        self.conv2 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=6, padding='same', activation='relu')

        # Аналогично предыдущему, но dilation_rate=12 — еще больше расширяет поле обзора,
        # чтобы сеть могла видеть более крупный контекст
        self.conv3 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=12, padding='same', activation='relu')

        # Еще более расширенное поле обзора — dilation_rate=18,
        # это помогает захватывать многоуровневые масштабные признаки изображения
        self.conv4 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=18, padding='same', activation='relu')

        # Финальный 1x1 свертка, чтобы объединить все полученные признаки в 256 каналов,
        # снова с активацией ReLU для усиления выражения признаков
        self.conv5 = tf.keras.layers.Conv2D(256, (1, 1), padding='same', activation='relu')

    def call(self, inp, is_training=False):
        # Применяем все четыре сверточных слоя к входу независимо, получая 4 разных набора признаков
        out1 = self.conv1(inp)
        out2 = self.conv2(inp)
        out3 = self.conv3(inp)
        out4 = self.conv4(inp)

        # Объединяем эти 4 набора признаков по канальному измерению (axis=3 — последний размер),
        # таким образом сеть может использовать информацию с разных масштабов
        out = tf.concat([out1, out2, out3, out4], axis=3)

        # Пропускаем объединённые признаки через последний 1x1 слой, чтобы уменьшить размерность и улучшить представление
        out = self.conv5(out)
        return out


class ASPPNet(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # Несколько сверточных слоев с ядрами 3x3, padding='same' сохраняет пространственный размер,
        # количество фильтров растет, чтобы увеличить глубину и выраженность признаков,
        # активация ReLU — для нелинейного обучения сложных признаков
        self.conv1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')
        self.conv4 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')
        self.conv5 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv6 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv7 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        self.conv8 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        self.conv9 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        self.conv10 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')

        # 1x1 свертка с 48 фильтрами — уменьшает количество каналов для облегчения слияния признаков
        self.conv11 = tf.keras.layers.Conv2D(48, (1, 1), padding='same', activation='relu')

        # Два 3x3 слоя с 256 фильтрами для дообработки и улучшения признаков после слияния
        self.conv12 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv13 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')

        # Финальный 1x1 слой с одним фильтром для вывода карты сегментации,
        # без активации, так как функция активации (сигмоида) применяется позже
        self.conv14 = tf.keras.layers.Conv2D(1, (1, 1), padding='same', activation=None)

        # Макспулинг с ядром 2x2 и шагом 2x2 уменьшает ширину и высоту в 2 раза,
        # padding='same' округляет размер вверх при необходимости
        self.maxpool = tf.keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same')

        # ASPP блок для захвата признаков на нескольких масштабах в глубине сети
        self.aspp = ASPPBlock()

    def call(self, x):
        # Последовательно применяем сверточные слои и слои макспулинга
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.maxpool(out)  # уменьшаем размер

        out = self.conv3(out)
        out = self.conv4(out)
        out = self.maxpool(out)  # уменьшаем размер

        out = self.conv5(out)
        out = self.conv6(out)
        out_enc_mid = out  # сохраняем для пропуска (skip connection)

        out = self.maxpool(out)  # уменьшаем размер

        out = self.conv7(out)
        out = self.conv8(out)
        out = self.maxpool(out)  # уменьшаем размер

        out = self.conv9(out)
        out = self.conv10(out)

        # Применяем ASPP блок для объединения контекстной информации разных масштабов
        out = self.aspp(out)

        # Изменяем размер выхода ASPP блока до размера промежуточного признака для слияния
        # Используем билинейную интерполяцию для плавного масштабирования
        out = tf.image.resize(out, tf.shape(out_enc_mid)[1:3], tf.image.ResizeMethod.BILINEAR)

        # Уменьшаем число каналов сохраненного промежуточного признака для слияния
        out_enc_mid = self.conv11(out_enc_mid)

        # Объединяем по каналам (axis=3) два тензора: выход ASPP и уменьшенный промежуточный признак
        out = tf.concat([out, out_enc_mid], axis=3)

        # Два сверточных слоя для уточнения и дообработки объединённых признаков
        out = self.conv12(out)
        out = self.conv13(out)

        # Финальный 1x1 слой для получения одноканальной карты (например, маски сегментации)
        out = self.conv14(out)

        # Масштабируем карту обратно к исходному размеру входного изображения
        out = tf.image.resize(out, tf.shape(x)[1:3], tf.image.ResizeMethod.BILINEAR)

        # Применяем сигмоидальную активацию, чтобы получить вероятности (от 0 до 1)
        # Обычно для бинарной сегментации (фон/объект)
        out = tf.nn.sigmoid(out)

        return out

# Создание экземпляра модели, которая готова к обучению или предсказанию
model = ASPPNet()
